In [89]:
import os
import io
import time
import sqlite3

In [90]:
iterations = 1000

In [98]:
# TEST MULTIPLE SINGLE-INSERTS
# this option is the second fastest when the database is empty
# NOTE: i have seen this method outperform the executemany method on rare occasions
if os.path.exists('/repostdb.sqlite'):
    os.remove('/repostdb.sqlite')

conn = sqlite3.connect('./repostdb.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS Posts (Date INT, Content TEXT, Url TEXT, Location TEXT, Author TEXT, Title TEXT);')
conn.commit()
c.close()

start = time.time()
conn = sqlite3.connect('./repostdb.sqlite')
c = conn.cursor()

for i in range(iterations):
    c.execute('INSERT INTO Posts (Date, Content, Url, Location, Author, Title) VALUES (?, ?, ?, ?, ?, ?);',
        (24102018, 'CONTENT', 'URL', 'LOCATION', 'AUTHOR', 'TITLE'))
    
conn.commit()
c.close()
print("MULTIPLE SINGLE-INSERTS ELAPSED TIME: ", time.time() - start)

MULTIPLE SINGLE-INSERTS ELAPSED TIME:  0.0063800811767578125


In [99]:
# TEST SINGLE MULTIPLE-INSERT
# this option is the slowest when the database is empty
if os.path.exists('/repostdb.sqlite'):
    os.remove('/repostdb.sqlite')

conn = sqlite3.connect('./repostdb2.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS Posts (Date INT, Content TEXT, Url TEXT, Location TEXT, Author TEXT, Title TEXT);')
conn.commit()
c.close()

# begin test for multiple single inserts  
start = time.time()
conn = sqlite3.connect('./repostdb.sqlite')
c = conn.cursor()

rows = []
for i in range(iterations):
    rows.append((24102018, 'CONTENT', 'URL', 'LOCATION', 'AUTHOR', 'TITLE'))

values = ', '.join(map(str, rows))
c.execute('INSERT INTO Posts (Date, Content, Url, Location, Author, Title) VALUES {}'.format(values))
    
conn.commit()
c.close()
print("SINGLE MULTIPLE-INSERT ELAPSED TIME: ", time.time() - start)

SINGLE MULTIPLE-INSERT ELAPSED TIME:  0.010954141616821289


In [100]:
# TEST EXECUTEMANY
# this option is the fastest when the database is empty
if os.path.exists('/repostdb.sqlite'):
    os.remove('/repostdb.sqlite')

conn = sqlite3.connect('./repostdb.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS Posts (Date INT, Content TEXT, Url TEXT, Location TEXT, Author TEXT, Title TEXT);')
conn.commit()
c.close()

# begin test for multiple single inserts  
start = time.time()
conn = sqlite3.connect('./repostdb.sqlite')
c = conn.cursor()

rows = []
for i in range(iterations):
    rows.append((24102018, 'CONTENT', 'URL', 'LOCATION', 'AUTHOR', 'TITLE'))

c.executemany("INSERT INTO Posts (Date, Content, Url, Location, Author, Title) VALUES (?, ?, ?, ?, ?, ?)", rows)

conn.commit()
c.close()
print("EXECUTEMANY ELAPSED TIME: ", time.time() - start)

EXECUTEMANY ELAPSED TIME:  0.0039370059967041016


In [101]:
# ===========================================================================
# now we will do the same test, but with 100,000 rows already in the database
# pls excuse this copy-paste job
# ===========================================================================
pop_iterations = 100000

In [102]:
# TEST MULTIPLE SINGLE-INSERTS WITH POPULATED DB
# this option is the second fastest when the database is populated
if os.path.exists('/repostdb.sqlite'):
    os.remove('/repostdb.sqlite')

conn = sqlite3.connect('./repostdb.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS Posts (Date INT, Content TEXT, Url TEXT, Location TEXT, Author TEXT, Title TEXT);')
# FILL THAT DB BBY
for i in range(pop_iterations):
    c.execute('INSERT INTO Posts (Date, Content, Url, Location, Author, Title) VALUES (?, ?, ?, ?, ?, ?);',
        (24102018, 'CONTENT', 'URL', 'LOCATION', 'AUTHOR', 'TITLE'))
conn.commit()
c.close()

start = time.time()
conn = sqlite3.connect('./repostdb.sqlite')
c = conn.cursor()

for i in range(iterations):
    c.execute('INSERT INTO Posts (Date, Content, Url, Location, Author, Title) VALUES (?, ?, ?, ?, ?, ?);',
        (24102018, 'CONTENT', 'URL', 'LOCATION', 'AUTHOR', 'TITLE'))
    
conn.commit()
c.close()
print("MULTIPLE SINGLE-INSERTS ELAPSED TIME: ", time.time() - start)

MULTIPLE SINGLE-INSERTS ELAPSED TIME:  0.006088733673095703


In [103]:
# TEST SINGLE MULTIPLE-INSERT WITH POPULATED DB
# this option is the slowest when the database is populated
if os.path.exists('/repostdb.sqlite'):
    os.remove('/repostdb.sqlite')

conn = sqlite3.connect('./repostdb2.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS Posts (Date INT, Content TEXT, Url TEXT, Location TEXT, Author TEXT, Title TEXT);')
for i in range(pop_iterations):
    c.execute('INSERT INTO Posts (Date, Content, Url, Location, Author, Title) VALUES (?, ?, ?, ?, ?, ?);',
        (24102018, 'CONTENT', 'URL', 'LOCATION', 'AUTHOR', 'TITLE'))
conn.commit()
c.close()

# begin test for multiple single inserts  
start = time.time()
conn = sqlite3.connect('./repostdb.sqlite')
c = conn.cursor()

rows = []
for i in range(iterations):
    rows.append((24102018, 'CONTENT', 'URL', 'LOCATION', 'AUTHOR', 'TITLE'))

values = ', '.join(map(str, rows))
c.execute('INSERT INTO Posts (Date, Content, Url, Location, Author, Title) VALUES {}'.format(values))
    
conn.commit()
c.close()
print("SINGLE MULTIPLE-INSERT ELAPSED TIME: ", time.time() - start)

SINGLE MULTIPLE-INSERT ELAPSED TIME:  0.00785684585571289


In [104]:
# TEST EXECUTEMANY WITH POPULATED DB
# this option is the fastest when the database is populated
if os.path.exists('/repostdb.sqlite'):
    os.remove('/repostdb.sqlite')

conn = sqlite3.connect('./repostdb.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS Posts (Date INT, Content TEXT, Url TEXT, Location TEXT, Author TEXT, Title TEXT);')
for i in range(pop_iterations):
    c.execute('INSERT INTO Posts (Date, Content, Url, Location, Author, Title) VALUES (?, ?, ?, ?, ?, ?);',
        (24102018, 'CONTENT', 'URL', 'LOCATION', 'AUTHOR', 'TITLE'))
conn.commit()
c.close()

# begin test for multiple single inserts  
start = time.time()
conn = sqlite3.connect('./repostdb.sqlite')
c = conn.cursor()

rows = []
for i in range(iterations):
    rows.append((24102018, 'CONTENT', 'URL', 'LOCATION', 'AUTHOR', 'TITLE'))

c.executemany("INSERT INTO Posts (Date, Content, Url, Location, Author, Title) VALUES (?, ?, ?, ?, ?, ?)", rows)

conn.commit()
c.close()
print("EXECUTEMANY ELAPSED TIME: ", time.time() - start)

EXECUTEMANY ELAPSED TIME:  0.0033872127532958984
